<a href="https://colab.research.google.com/github/pomoq-dev/video-clipping/blob/master/notebooks/download_split_shuffle_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

SUBLICP_LEN = 30
SKIP_FIRST = 11
ORIGINAL_AUDIO = False
CLIPS_IN_VIDEO = 30

from google.colab import drive
import os

drive.mount('/content/drive')

DIR_DOWNLOAD_VIDEO = '/content/drive/MyDrive/DOWNLOAD_JOKE_VIDEO'
DIR_SPLIT_FRAGMENTS = '/content/drive/MyDrive/DOWNLOAD_JOKE_VIDEO_FRAGMENTS'
DIR_REACTIONS = '/content/drive/MyDrive/REACTIONS_VIDEO'
RES_DIR = '/content/drive/MyDrive/RES_FRAGMENTS_WITH_REACTIONS'

os.makedirs(DIR_DOWNLOAD_VIDEO, exist_ok=True)
os.makedirs(DIR_SPLIT_FRAGMENTS, exist_ok=True)
os.makedirs(RES_DIR, exist_ok=True)

In [ ]:
video_urls = ['https://www.youtube.com/watch?v=jz6mTXV4AMc',
              'https://www.youtube.com/watch?v=ug9BcMvUHQU',
              'https://www.youtube.com/watch?v=xRhWqISvhKU',
              'https://www.youtube.com/watch?v=a0URj5SoMOc',
              'https://www.youtube.com/watch?v=3KByoovkDL4',
              'https://www.youtube.com/watch?v=n0vheTRoHVM',
              'https://www.youtube.com/watch?v=6wEOOFXfjZ4',
              'https://www.youtube.com/watch?v=EKkYwu7nYLc']

In [ ]:
print('Git clone project and install requirements...')
!git clone https://github.com/pomoq-dev/video-clipping.git &> /dev/null
%cd video-clipping/

!export PYTHONPATH=/content/video-clipping:$PYTHONPATH
!wget https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -O /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

!pip install -r requirements.txt

In [ ]:
from vidlib import download_and_split_video
download_and_split_video.download_and_split_by_clips(video_urls, DIR_DOWNLOAD_VIDEO, DIR_SPLIT_FRAGMENTS, subclip_len=SUBLICP_LEN, skip_first=SKIP_FIRST, add_audio=ORIGINAL_AUDIO)

In [ ]:
from vidlib import files_tools, video_tools
from moviepy import editor as mpe

reactions_paths = files_tools.list_full_paths(DIR_REACTIONS, only_format='.mp4')

clips_paths = files_tools.list_full_paths(DIR_SPLIT_FRAGMENTS, only_format='.mp4')
random.shuffle(clips_paths)
clips_chunks = [clips_paths[i:i + CLIPS_IN_VIDEO] for i in range(0, len(clips_paths), CLIPS_IN_VIDEO)]

for res_ind, clips_chunk in enumerate(clips_chunks):
    chunk_with_reactions = []
    for clip_path in clips_chunk:
        clip = mpe.VideoFileClip(clip_path)
        width = clip.size[0]
        chunk_with_reactions.append(clip)
        reaction_path = random.choice(reactions_paths)
        reaction = mpe.VideoFileClip(reaction_path)
        reaction = reaction.resize(width=width)
        chunk_with_reactions.append(reaction)
    video = video_tools.concatenate_videos(chunk_with_reactions)
    video_tools.save_video(video, os.path.join(RES_DIR, 'res{}.mp4'.format(res_ind)))
